# LISST-100 data processing notebook

The following notebook was developed by the Suspended Material and Seabed Monitoring and Modelling group (SUMO) from the Institute of Natural Sciences, Belgium. It aims at providing a standard protocol to process data collected in-situ with a LISST-100 instrument placed on a rosette profiling the water column. To use this notebook, simply import the data previously converted to asc by the LISST-SOPv5 software, run the cells containing code and inspect the results obtained. The graphs can then be saved as jpg and the processed data as csv.

In this notebook, the LISST-100 data processing has been divided into three stages:
1. *ASC data generation*: The first stage, during which the raw data is converted from .dat to .asc, requires the use of the LISST-SOPv5 software. 
2. *Data import & flagging*: This notebook can then be used to carry out the second stage, which includes the import and pre-processing of the data (i.e. date conversion, data flagging). At the end of this stage, the user can save the full dataset with a flagging system and dates as datetime. 
3. *Statistics computation & visualization*: The third stage performs the calculation of different statistical parameters on a clean dataset (after filtrating data above a certain flag value), such as mean particle diameter, D10, D50, D90 and distribution characteristics, and displays outputs as graphs. The clean dataset as well as the graphs can be saved. Data are also merged per cast, allowing the user to save a csv containing mean values of each cast.

**Important!** Before starting the processing, make sure that all the necessary packages and libraries are installed on your computer and that you run the cell below to import everything that is required.
**Important!!** The beam attenuation is computed assuming the path length is 50 mm and is not automatically adapted when using a path reduction module! This option will be added in a further version of this code. As indicated in the LISST-100 user manual, these are the expressions used to convert:
- c = -ln(optical transmission)x(1/0.025 m) if you are using a 50% PRM (25 mm path length)
- c = -ln(optical transmission)x(1/0.01 m) if you are using an 80% PRM (10 mm path length) 
- c = -ln(optical transmission)x(1/0.005 m) if you are using a 90% PRM (5 mm path length)

In [ ]:
                                                    ## Cell 01 ##
import os
import subprocess
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter.filedialog import asksaveasfilename
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import seaborn as sns
                                                        ###

## Stage 1: Conversion of dat files to asc files

In this stage, the conversion of the raw LISST-100 data is performed using the LISST-SOPv5 software, which can be downloaded freely on https://www.sequoiasci.com/product/lisst-100x/. The different steps to be perfomed are explained in more details in this section. 

1. Open .dat file with corresponding background file (.asc) and select output directory. Once that is done, a graph will appear showing the different rings (from 1 to 32) through time, as well as another graph showing the auxiliary parameters (if any) through time. In your output directory, you will notice that new files have been added: a .psd and a .log files (for both round particles and random shapes (rs)).

2. After inspection of the data, you can then click on the "Process File" button. Be aware that depending on the size of your data, that operation might take a long time. 

## Stage 2: Data pre-processing

#### Stage 2 - Step 1 : Enter metadata

In this cell, the user will be asked to enter the campaign identification code, year at which the data were collected, the name of the sampling location and coordinates in decimal degrees (XX.XXXXX). Press enter after each input.
**Important!** Avoid using "/" in the campaign identification code, as it might create issues when saving the data.

In [ ]:
                                                    ## Cell 02 ##
campaign_code = input("Enter the campaign identification code: ")
year = int(input("Enter the year of the data: "))
location = input("Enter the name of the sampling location: ")
latitude = input("Enter the latitude of the data: ")
longitude = input("Enter the longitude of the data: ")
                                                        ###

#### Stage 2 - Step 2 : Data import & formatting

Upon running this cell, a new window should automatically open and invite the user to select the asc file to be processed. Be aware that this window might open in the background and could be hidden under the current window. Once imported, the data will be attributed column names, campaign identification number, sampling location and year (as inserted by the user in the cell above). In addition to that, day_hour and minute_second columns will be converted to jday and datetime values. A new column containing a flag value (set by default to zero at this step) will be added at the end of the dataset.

The dataset will be displayed upon running the cell, enabling the user to check that columns and values were assigned correctly and that the calculated date and time of the data correspond to the actual sampling time.

In [ ]:
                                                    ## Cell 03 ##
# Open a window to select input file
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename(title="Select an ASC file", filetypes=[("ASC files", "*.asc")])
directory_path = os.path.dirname(file_path)

output_directory = f"{directory_path}/LISST-100-{campaign_code}-{location}-processed"
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)
    print(f'Existing output folder removed at: {output_directory}')
os.makedirs(output_directory)
print(f'Output folder created at: {output_directory}')

if file_path:  
    try:
        data = pd.read_csv(file_path, header=None, sep=" ")
        print("File successfully loaded!") 
        data = data.iloc[:, 0:-1]
        # Add column names
        data.columns = ["2.72","3.2","3.78","4.46","5.27","6.21","7.33","8.65","10.2","12.1","14.2","16.8","19.8","23.4","27.6","32.5","38.4","45.3","53.5","63.1","74.5","87.9","104","122","144","170","201","237","280","331","390","460","laser_transmission_sensor_mW","battery_voltage_V","external_input_1_mW","laser_reference_sensor_mW","pressure_m","temperature_C","day_hour","minute_second","optical_transmission","beam_attenuation_m"]
        # Add campaign identification number, sampling location name, latitude, longitude and year
        data['campaign'] = campaign_code
        data['location'] = location
        data['latitude'] = latitude
        data['longitude'] = longitude
        data['year'] = year
        # Calculate datetime
        data['jday'] = round((data['day_hour'] / 100),0)
        start_of_year = datetime(year, 1, 1)
        data['datetime'] = pd.to_datetime(data['jday'].apply(lambda x: start_of_year + timedelta(days=x-1))) \
    + pd.to_timedelta((data['day_hour'] % 100), unit='h') \
    + pd.to_timedelta(data['minute_second'] // 100, unit='m') \
    + pd.to_timedelta(data['minute_second'] % 100, unit='s')
        data['flag'] = 0
        # Compute sampling start and stop datetime
        sampling_start = data['datetime'].min()
        sampling_stop = data['datetime'].max()
        # Display output
        print(f'Data were collected at {location} between {sampling_start} and {sampling_stop} during campaign {campaign_code}.')
        display(data)
    except Exception as e:
        print(f"Error reading the file: {e}")
else:
    print("No file selected")
                                                        ###

#### Stage 2 - Step 3: Flagging of the data

In this cell, flagging operations are performed as follows:
1. A first check is done on the **reference laser**, if values are below 0.2 mW, this indicates that the laser is likely not working properly. Quality flags will be assigned to 1 if data are above this value and to 3 if not (see flags meaning below). If more than 10% of the data present such a low value, a warning message will be displayed.  
2. Data out of the water are flagged with the value of 4 (rows where the beam attenuation is equal to zero). The **beam attenuation** corresponds to the loss of light intensity when a laser beam passes through water, due to both absorption and scattering. Higher attenuation values typically indicate more turbid water. 
3. The **laser transmission** values are checked. This indicates how much of the laser light has passed through water without being absorbed or scattered. A value of 100% indicates that there has been no light loss through the sample (either water without particles or air) while values below indicate scattering and absorption (which is expected in seawater). Values above 100%, however, indicate a sensor malfunction or miscalibration. In this cell, a first inspection of the data calculates how many data are above 100% and if that number is acceptable, these rows are removed from the data. If most data (>50%) are above 100%, then a warning message is generated. Data above 100 are flagged with the value of 3. 
4. The **optical transmission** is checked: values should be ranging between 0 and 1. Values outside that range should be discarded (flag 4). However, values above 0.98-0.995 reflect extremely clear water conditions, meaning a low signal-to-noise ratio (flag 3, to be taken with caution. Above that range, values should be discarded), whereas values below 0.10 reflect very turbid data and should be discarded. 
5. Finally, a flag of 3 is attributed to **outliers** detected based on the total volume concentration calculated.

Once the flagging has been performed, a graph allows the user to visualize the quality of the data. The complete flagged dataset is automatically saved in the output directory selected by the user at the beginning of this notebook. In stage 3, a cell then allows to filter out all the data with a quality flag equal or higher than 4.

Quality flags are defined following the quality flags standards defined by the NERC Environmental Data Service of the British Oceanographic Data Centre (https://vocab.nerc.ac.uk/collection/L20/current/):
0: No quality control
1: Good value
2: Probably good value
3: Probably bad value
4: Bad value
5: Changed value
6: Value below detection
7: Value in excess
8: Interpolated value
9: Missing value

In [ ]:
                                                    ## Cell 04 ##
# Step 1: Flagging based on reference_laser
data['flag'] = data.apply(lambda row: max(row['flag'], 1) if row['laser_reference_sensor_mW'] > 0.02 else max(row['flag'], 3), axis=1)
reference_below_thsld = (data['laser_reference_sensor_mW'] <= 0.02).sum() / len(data) * 100
if reference_below_thsld > 20:  # If more than 20% of the data are below 0.02 mW
    messagebox.showwarning("Laser should be checked", 
                           f"Warning: {reference_below_thsld:.0f}% of the data have a laser reference value below 0.02 mW. "
                           "Please check laser.")

# Step 2: Beam attenuation flagging
data['flag'] = data.apply(lambda row: max(row['flag'], 4) if row['beam_attenuation_m'] <= 0 else row['flag'], axis=1)

# Step 3: Laser transmission flagging
data['flag'] = data.apply(lambda row: 3 if row['laser_transmission_sensor_mW'] > 100 and row['flag'] < 3 else row['flag'], axis=1)
underwater_data = data[data['beam_attenuation_m'] > 0]
above_100_percent = (underwater_data['laser_transmission_sensor_mW'] > 100).sum() / len(underwater_data) * 100
above_100_percent = (data['laser_transmission_sensor_mW'] > 100).sum() / len(data) * 100
if above_100_percent > 50:  # If more than 50% of the data are above 100%
    messagebox.showwarning("Recalibration Needed", 
                           f"Warning: {above_100_percent:.0f}% of the underwater data are above 100% transmission. "
                           "Please recalibrate the sensor.")
    
# Step 4: Optical transmission flagging
data['flag'] = data.apply(lambda row: 3 if row['optical_transmission'] > 0.98 and row['optical_transmission'] < 0.995 and row['flag'] < 3 else row['flag'], axis=1)
data['flag'] = data.apply(lambda row: 4 if row['optical_transmission'] >= 0.995 and row['flag'] < 4 else row['flag'], axis=1)
data['flag'] = data.apply(lambda row: 4 if row['optical_transmission'] <= 0.10 and row['flag'] < 4 else row['flag'], axis=1)
    
# Step 5: Outlier detection using IQR
volume_columns = ['2.72', '3.2', '3.78', '4.46', '5.27', '6.21', '7.33', '8.65', 
                  '10.2', '12.1', '14.2', '16.8', '19.8', '23.4', '27.6', '32.5', 
                  '38.4', '45.3', '53.5', '63.1', '74.5', '87.9', '104', '122', 
                  '144', '170', '201', '237', '280', '331', '390', '460']

data['total_volume_concentration'] = data[volume_columns].sum(axis=1)
Q1 = data['total_volume_concentration'].quantile(0.25)
Q3 = data['total_volume_concentration'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
data['flag'] = data.apply(lambda row: 3 if row['total_volume_concentration'] < lower_bound or row['total_volume_concentration'] > upper_bound else row['flag'], axis=1)
data['is_outlier'] = (data['total_volume_concentration'] < lower_bound) | (data['total_volume_concentration'] > upper_bound)
outlier_count = data['is_outlier'].sum()
print(f'Number of outliers detected: {outlier_count}')
    
# Warning message quality flags
percentage_high_flags = (data['flag'] >= 3).sum() / len(data) * 100
if percentage_high_flags > 75:  # If more than 75% of the data have a quality flag of 3 or higher
    messagebox.showwarning("Unsatisfactory data quality", 
                           f"Warning: {percentage_high_flags:.0f}% of the data have a quality flag above 3. "
                           "Data should be used with caution.")
    
# Output
print('Flagging has been successfully performed on the complete dataset:')
display(data)

flag_counts = data['flag'].value_counts().sort_index()
colors = {1: 'green', 2: 'yellow', 3:'orange', 4: 'red'}
fig, ax = plt.subplots()
flag_counts.plot(kind='bar', color=[colors.get(flag, 'blue') for flag in flag_counts.index], ax=ax)
ax.grid(axis='both', which='both', linewidth=0.3)
ax.set_title("Count per flag category")
ax.set_xlabel("Flag")
ax.set_ylabel("Count")
plt.show()

# Visualizing outliers
plt.figure(figsize=(8, 4))
plt.scatter(data.index, data['total_volume_concentration'], label='', color='blue', s=10, alpha=0.5)
plt.scatter(data[data['is_outlier']].index, data[data['is_outlier']]['total_volume_concentration'], 
            label='Outliers', color='red', marker='x', s=15)
plt.axhline(lower_bound, color='orange', linestyle='--', label='Lower limit')
plt.axhline(upper_bound, color='orange', linestyle='--', label='Upper limit')
plt.grid(axis='both', which='both', linewidth=0.3)
plt.title('Outlier detection')
plt.xlabel('')
plt.ylabel('Total volume concentration (µl/l)')
plt.legend()
plt.show()

data.drop(columns=['total_volume_concentration'], inplace=True)

# Save as csv
data_path = f"{output_directory}/{campaign_code}-{location}-LISST100-full-data.csv"
if data_path:
    data.to_csv(data_path, index=False)
    print(f"File saved to {file_path}")
else:
    print("Save operation cancelled.")
                                                        ###

## Stage 3: Statistics computation & visualization

#### Stage 3 - Step 1: Statistics computation

By running this cell, a series of statistics are calculated on the full dataset. This includes:
1. **Total volume concentration**: This sums up the volume concentration of each grain size class per row.
2. **Relative volume concentration**: This calculates the percentage that each class represents compared to the total volume concentration.
3. **Mean diameter**: This corresponds to the mean diameter of particles in each row weighted by their volume concentration and normalized by the total volume concentration.
4. **D10, D50, D90 values**: These represent the diameter below which 10%, 50% or 90% of the volume of the data are found. The D10 helps characterizing the finer fraction of the sample, the D50 corresponds to the median diameter and the D90 characterizes the coarser sediment fraction. These values are calculated based on the cumulative distribution.
5. **Span**: It is a measure of the sorting of particle sizes, as a normalized measure of the distribution spread around the median particle size, showing the relative range of the middle 80% of the particle size distribution. Small values indicate good sorting (close to 1) while a larger span indicates poor sorting of the particles.
6. **Standard deviation**: It measures the average dispersion of particle sizes from the mean, a greater standard deviation shows greater variability around the mean with possible outliers or tails. If the span and standard deviation are low, it shows a uniform distribution; if the span is moderate but the standard deviation is high, it shows a moderate spread with outliers; and if the span and standard deviation are high, it shows a wide distribution.
7. **Mode**: The mode is the particle size that has the largest volume of the distribution (peak in the distribution).
8. **Peaks in the distribution**: It identifies peaks in the particle size distribution.

The particle size distribution combined with the cumulative volume concentration is displayed upon running of this cell as well as a table with the mean value of all the calculated parameters. The graph and updated complete clean dataframe are automatically saved in the directory selected by the user.

In [ ]:
                                                    ## Cell 07 ##  
# Calculate the total volume concentration 
volume_concentration_columns = ['2.72', '3.2', '3.78', '4.46', '5.27', '6.21', '7.33', '8.65', 
                  '10.2', '12.1', '14.2', '16.8', '19.8', '23.4', '27.6', '32.5', 
                  '38.4', '45.3', '53.5', '63.1', '74.5', '87.9', '104', '122', 
                  '144', '170', '201', '237', '280', '331', '390', '460']
average_volume_fractions = data[volume_concentration_columns].mean()
data['total_volume_concentration_ul_l'] = data[volume_concentration_columns].sum(axis=1)

# Calculate the percentage of each class compared to the total volume concentration
for col in volume_concentration_columns:
    data[f'{col}_%'] = ((data[col] / data['total_volume_concentration_ul_l']) * 100).astype(float)

# Calculate mean diameter 
volume_concentrations = data[volume_concentration_columns].values
grain_sizes = np.array([float(col) for col in volume_concentration_columns])
data['mean_diameter_um'] = np.sum(volume_concentrations * grain_sizes, axis=1) / data['total_volume_concentration_ul_l']

# Cumulative volume concentration
cumulative_volumes = np.cumsum(data[volume_concentration_columns].values, axis=1)

# Percentiles calculation function
def calculate_d_percentile(cumulative_volumes, grain_sizes, total_volume, percentile):
    target = total_volume * (percentile / 100.0)
    d_percentile = []
    for i in range(len(total_volume)):
        if total_volume.iloc[i] == 0: 
            d_percentile.append(np.nan)
        else:
            greater_equal_idx = np.argmax(cumulative_volumes[i, :] >= target.iloc[i])
            d_percentile.append(grain_sizes[greater_equal_idx])
    return np.array(d_percentile)

# Calculate D10, D50, D90
data['D10_um'] = calculate_d_percentile(cumulative_volumes, grain_sizes, data['total_volume_concentration_ul_l'], 10)
data['D50_um'] = calculate_d_percentile(cumulative_volumes, grain_sizes, data['total_volume_concentration_ul_l'], 50)
data['D90_um'] = calculate_d_percentile(cumulative_volumes, grain_sizes, data['total_volume_concentration_ul_l'], 90)

# Calculate the span
data['span'] = (data['D90_um'] - data['D10_um']) / data['D50_um']

# Calculate the standard deviation
def calculate_weighted_std(grain_sizes, volume_concentrations, mean_diameter):
    variance = np.sum(volume_concentrations * (grain_sizes - mean_diameter[:, None])**2, axis=1) / np.sum(volume_concentrations, axis=1)
    return np.sqrt(variance)
data['std_dev_um'] = calculate_weighted_std(grain_sizes, volume_concentrations, data['mean_diameter_um'].values)

# Calculate the mode
def calculate_mode(grain_sizes, volume_concentrations):
    mode_values = []
    for vc in volume_concentrations:
        if np.all(vc == 0):
            mode_values.append(np.nan)
        else:
            mode_index = np.argmax(vc)
            mode_values.append(grain_sizes[mode_index])
    return np.array(mode_values)
data['mode_um'] = calculate_mode(grain_sizes, volume_concentrations)

# Identify peaks in the distribution 
def find_all_peaks(grain_sizes, volume_concentrations):
    all_peaks = []
    for vc in volume_concentrations:
        if np.all(vc == 0):
            all_peaks.append([])
        else:
            peaks, _ = find_peaks(vc)
            peak_sizes = grain_sizes[peaks]
            all_peaks.append(peak_sizes.tolist())
    return all_peaks
data['peaks'] = find_all_peaks(grain_sizes, volume_concentrations)

# Display updated dataframe
print('Statistics have been successfully computed')
display(data)

# Display the histogram of the particle size distribution and the cumulative volume distribution (in red)
total_volume_concentration_per_class = data[volume_concentration_columns].sum(axis=0)
fig, ax1 = plt.subplots(figsize=(10, 6), dpi=300)
ax1.plot(grain_sizes, total_volume_concentration_per_class, color='blue', marker='o', 
         label='Particle size distribution', linestyle='-')
ax1.set_xscale('log')
ax1.set_xlabel('Grain size (µm)')
ax1.set_ylabel('Total volume concentration (µl/l)')
ax1.set_title(f'LISST-100 {location} {campaign_code}')
ax1.grid(axis='both', which='both', linewidth=0.3)
cumulative_volumes = np.cumsum(total_volume_concentration_per_class)
ax2 = ax1.twinx() 
ax2.plot(grain_sizes, cumulative_volumes / cumulative_volumes[-1] * 100, color='red', 
         label='Cumulative Distribution', marker='o')
ax2.set_ylabel('Cumulative volume (%)', color='red')
ax2.tick_params(axis='y', labelcolor='red')
ax1.axvline(data['D10_um'].mean(), color='green', linestyle='--', label='D10')
ax1.axvline(data['D50_um'].mean(), color='orange', linestyle='--', label='D50')
ax1.axvline(data['D90_um'].mean(), color='purple', linestyle='--', label='D90')
ax1.legend(loc='upper left')
ax2.legend(loc='upper center')
plt.tight_layout()
plt.show()

# Display table with the mean of each parameter
mean_values = {
    'Total volume concentration (µl/l)': data['total_volume_concentration_ul_l'].mean(),
    'Mean diameter (µm)': data['mean_diameter_um'].mean(),
    'D10 (µm)': data['D10_um'].mean(),
    'D50 (µm)': data['D50_um'].mean(),
    'D90 (µm)': data['D90_um'].mean(),
    'Span': data['span'].mean(),
    'Standard deviation (µm)': data['std_dev_um'].mean(),
    'Mode (µm)': data['mode_um'].mean()
}
mean_values_df = pd.DataFrame(mean_values, index=[0])
print('Mean values of calculated parameters for the complete clean dataset:')
display(mean_values_df)

# Save output
graph_file_path = f"{output_directory}/{campaign_code}-{location}-LISST100-full-PSD.png"
fig.savefig(graph_file_path) 
print(f'Graph is saved to: {output_directory}')
                                                        ###

#### Stage 3 - Step 2: Removal of suspicious data & creation of a 'clean' dataset

This step allows the user to choose a certain flag threshold at and above which data are discarded for further analysis. The default value is set to 4, meaning all data flagged 4 and above will be removed from the filtered dataframe. In this cell, values considered outliers can also be removed.

In [ ]:
                                                    ## Cell 06 ##
try:
    threshold = int(input("Enter the threshold for flag filtration (default is 4): ") or 4)
except ValueError:
    print("Invalid input, defaulting to threshold = 4.")
    threshold = 4
    
outliers = input("Remove outliers (yes or no): ")

# Filter out rows where flag is greater than or equal to the chosen threshold
filtered_data = data[data['flag'] < threshold]

# Filter out ouliers
if outliers == 'yes' or outliers == 'Yes':
    filtered_data = filtered_data[filtered_data['is_outlier'] != True]
elif outliers == 'no' or outliers == 'No':
    filtered_data = filtered_data
else:
    print("Invalid input, outliers were not filtered out.")

# Display the filtered data
print(f"After filtering, the dataset is:")
display(filtered_data)

# Save filtered data as csv
filtered_data_path = f"{output_directory}/{campaign_code}-{location}-LISST100-clean-data.csv"
filtered_data.to_csv(filtered_data_path, index=False)
                                                        ###

#### Stage 3 - Step 3: Division of the clean dataset into casts

In this cell, a cast letter will be attributed to each set of consecutive values separated from the previous set by at least 20 minutes. If needed, this time difference can be adapted in the code directly (line 8). If the user has other data for these casts, we advise comparing their timings to the table generated in this cell.

In [ ]:
                                                    ## Cell 08 ##
data = filtered_data if not filtered_data.empty else data

current_cast = 'A'
cast_numbers = [current_cast]

# Loop through the dataset and check the time difference between consecutive rows
for i in range(1, len(data)):
    time_diff = data['datetime'].iloc[i] - data['datetime'].iloc[i - 1]
    # If the time difference is greater than or equal to 20 minutes, increment the cast value
    if time_diff >= timedelta(minutes=10):
        current_cast = chr(ord(current_cast) + 1)
    cast_numbers.append(current_cast)

# Add the cast numbers as a new column in the dataframe
data.loc[:, 'cast'] = cast_numbers

# Group by the cast to find the start and stop time for each cast
summary_table = data.groupby('cast').agg(
    datetime_start=('datetime', 'min'),
    datetime_stop=('datetime', 'max')
).reset_index()

# Display the summary table
print('Summary of casts timings (clean data):')
display(summary_table)
                                                        ###

#### Stage 3 - Step 4: Comparison of the different casts

This cell generates a table and several graphs comparing the different casts of the processed dataset. These are saved in the directory previously selected by the user, together with a csv containing the mean values of each cast.

In [ ]:
                                                    ## Cell 09 ##
# Create summary table per cast with mean values
summary_table = data.groupby('cast').agg(
    datetime_start=('datetime', 'min'),
    datetime_stop=('datetime', 'max'),
    mean_total_volume_concentration=('total_volume_concentration_ul_l', 'mean'),
    mean_diameter=('mean_diameter_um', 'mean'),
    mean_D10=('D10_um', 'mean'),
    mean_D50=('D50_um', 'mean'),
    mean_D90=('D90_um', 'mean'),
    mean_span=('span', 'mean'),
    mean_std_dev=('std_dev_um', 'mean'),
    mean_mode=('mode_um', 'mean'),
).reset_index()
summary_table_path = f"{output_directory}/{campaign_code}-{location}-LISST100-cast-summary.csv"
summary_table.to_csv(summary_table_path, index=False)
print(f"Summary table saved to: {summary_table_path}")

unique_casts = data['cast'].unique()
colors = plt.cm.viridis(np.linspace(0, 1, len(unique_casts)))

# Display the summary table
print("Statistics per cast:")
display(summary_table)

# Plot the relative PSD for each cast
plt.figure(figsize=(10, 6), dpi=300)
volume_columns_percentage = [
    '2.72_%', '3.2_%', '3.78_%', '4.46_%', '5.27_%', 
    '6.21_%', '7.33_%', '8.65_%', '10.2_%', '12.1_%', 
    '14.2_%', '16.8_%', '19.8_%', '23.4_%', '27.6_%', 
    '32.5_%', '38.4_%', '45.3_%', '53.5_%', '63.1_%', 
    '74.5_%', '87.9_%', '104_%', '122_%', '144_%', 
    '170_%', '201_%', '237_%', '280_%', '331_%', 
    '390_%', '460_%'
]
for i, cast in enumerate(unique_casts):
    cast_data = data[data['cast'] == cast]
    volume_concentration_percentage = cast_data[volume_columns_percentage].mean(axis=0).values
    plt.plot(grain_sizes, volume_concentration_percentage, label=f'Cast {cast}', color=colors[i])
plt.title(f'LISST-100 {location} {campaign_code}: PSD per cast')
plt.xlabel('Particle size (µm)')
plt.ylabel('Relative volume (%)')
plt.legend(title='Relative PSD per cast')
plt.grid(axis='both', which='both', linewidth=0.3)
plt.xscale('log') 
plt.tight_layout()
psd_plot_path = f"{output_directory}/{campaign_code}-{location}-LISST100-PSD-per-cast.png"
plt.savefig(psd_plot_path, bbox_inches='tight')
plt.show()

# Plot comparison of different parameters between casts
parameters = ['mean_diameter_um', 'D10_um', 'D50_um', 'D90_um', 'span', 'mode_um']
plt.figure(figsize=(14, 10))
for i, param in enumerate(parameters):
    plt.subplot(2, 3, i + 1) 
    sns.boxplot(data=data, x='cast', y=param)
    plt.title(f'Boxplot of {param}')
    plt.xlabel('Cast')
    plt.ylabel(param)
    plt.grid(axis='both', which='both', linewidth=0.3)
plt.tight_layout()
psd_plot2_path = f"{output_directory}/{campaign_code}-{location}-LISST100-cast-comparison.png"
plt.savefig(psd_plot2_path, bbox_inches='tight')
plt.show()
                                                        ###